# Estimation-Maximization (EM) Algorithm

In [118]:
import numpy as np

## 0. Constants

In [119]:
N_GAUSSIANS = 5  # Number of Gaussian distributions.
N_SAMPLES = 1000  # Number of samples.

## 1. Dataset

Define helper functions.

In [120]:
def generateRandomCovarianceMatrix(n):
    """Generate a random covariance matrix.

    To generate a covariance matrix, we need to generate a random matrix Q, then
    Q @ Q.T would be a covariance matrix. This method works because Q @ Q.T has
    non-negative diagnonal elements, and it is symmetric and positive
    semidefinite.

    Args:
        n (int): number of dimensions.

    Return (numpy.ndarray ((n, n) & numpy.float64)): a random covariance matrix.
    """
    
    Q = np.random.default_rng().uniform(low=-1, high=1, size=(n, n))
    return Q @ Q.T

def multinoulli(p):
    """Draw one sample from a multinoulli distribution.

    Args:
        p (numpy.ndarray ((n,))): probability mass funtion.

    Return (int): an integer in [0, n) that represents a category.
    """

    # Retrieve the number of categories.
    n = p.shape[0]

    # Normalize the probabilities so that they sum to 1.
    p_normalized = p / p.sum()

    # Sampling.
    rn = np.random.default_rng().uniform(low=0, high=1)
    cummulated_mass = 0
    for i in range(n - 1):
        cummulated_mass += p_normalized[i]
        if rn < cummulated_mass:
            return i
    return n - 1

Generate the probabilities of selecting each Gaussian distribution.

In [121]:
# numpy.ndarray ((N_GAUSSIANS,) & numpy.float64)
p = np.random.default_rng().uniform(size=N_GAUSSIANS)

# Normalize the probabilities so that they sum to 1.
p /= p.sum()

Generate the means of Gaussian distributions.

In [122]:
# numpy.ndarray ((N_GAUSSIANS, 2) & numpy.float64)
means = np.random.default_rng().uniform(low=-5, high=5, size=(N_GAUSSIANS, 2))

Generate the covariance matrices of Gaussian distributions.

In [123]:
covs = []
for i in range(N_GAUSSIANS):
    covs.append(generateRandomCovarianceMatrix(2))

# numpy.ndarray ((N_GAUSSIANS, 2, 2) & numpy.float64)
covs = np.stack(covs)